In [ ]:
# imports
import os
import sys 
import random
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors 
import matplotlib.cbook as cbook 
plt.rcParams.update({'font.size': 12}) 
import mpld3 
import ast
#import ROOT
import uproot3 as ur3
import uproot as ur4
import pandas as pd 
from tqdm import tqdm 
from scipy.optimize import curve_fit 
from scipy import stats
from scipy.signal import lfilter, filtfilt
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import scipy.integrate as integrate
from lmfit import Model
import threading
from pathlib import Path
from matplotlib import gridspec
import datetime
import matplotlib.ticker as mticker
import time
import pathlib
import awkward as ak
import pprint
import pickle
import struct
from os.path import exists

In [ ]:
global Settings
Settings = {}

In [ ]:
def GetBranchFromTree(filename,branchname):
    myFile = ur4.open(filename)
    
    array = np.array(myFile[Settings["General"]["Treename"]][branchname].array()[0])

    myFile.close()
        
    return array

def checktype(value):
    if value.startswith('['):
        return ast.literal_eval(value)

    if value.isnumeric():
        return int(value)

    if '.' in value:
        splits = value.split('.',-1)
        if len(splits) == 2 and splits[0].isnumeric() and splits[1].isnumeric():
            return float(value)
        else:
            return value

    if '-' in value:
        splits = value.split('-',-1)
        if len(splits)>1 and splits[0]!='':
            return value
        else: 
            if '.' in value:
                return float(value)
            else:
                return int(value)
    return value

def LoadSettings(setfile):
    #Load Settings file
    print("Using "+setfile+" as Settings")
    with open(setfile, "r") as open_file:
        for line in open_file.readlines():
            if line.startswith('#') or len(line)<=1 or line.isspace():
                continue
            elif line.startswith('+'):  
                upper_key = line.strip("+").strip('\n')
            else:
                [lower_key,str_value] = line.split(" = ")
                value = checktype(str_value.strip('\n'))
                if upper_key not in Settings:
                    Settings.update({upper_key:{lower_key:value}})
                else:
                    Settings[upper_key].update({lower_key:value})
    open_file.close()

def GetFileList():
    lines = []
    path_in = Settings["General"]["Path_In"].replace("/mnt/s/","../../")
    
    if os.path.exists(path_in):
        with open(path_in, 'r') as file:
            for line in file:
                stripped_line = line.strip()
                if stripped_line:  # Check if the line is not empty
                    file_path = os.path.join(path_in, stripped_line, Settings["General"]["File_Name"]).replace("/mnt/s/","../../")
                    lines.append(file_path)
    else:
        print(f"The specified path '{path_in}' does not exist.")
    
    return lines

def Exponential_Decay(x, x0, tau):
    return np.exp(-(x-x0)/tau)

def Exponential_Decay_X(x,*params):
    sum_value = 0
    for i in range(0,int(len(params)),3):
        sum_value += params[2+i] * Exponential_Decay(x, params[i], params[1+i])
    return sum_value

In [ ]:
# First load settings
LoadSettings("./Settings.txt")
pprint.pprint(Settings)

file_list = GetFileList()
print(file_list)

combined_data = {}

for file in file_list:
    if os.path.exists(file):
        for i_ch in range(0,Settings["General"]["Number of Channels"]):
            data = GetBranchFromTree(file,Settings["General"]["Branchname"]+str(i_ch))
            if i_ch in combined_data:
                combined_data[i_ch] = np.append(combined_data[i_ch],data)
            else:
                combined_data.update({i_ch: data})
    else:
        print("{} does not exist!".format(file))

for key,data in combined_data.items():
    if len(data)<10:
        continue
    else:
        data = data[data > -100]
        data = data[data < 150]
    f1 = plt.figure(key ,figsize=[10,10], facecolor='white')
    plt.title(key)

    hist, bins = np.histogram(data,bins=np.arange(min(data),max(data)+Settings["Data"]["Bin Width"],Settings["Data"]["Bin Width"]))
    x_data = np.array((bins[:-1] + bins[1:]) / 2)
    y_data = np.array(hist)

    mask = (x_data >= Settings["Data"]["Fit Start"]) & (x_data <= Settings["Data"]["Fit End"])
    x_data = x_data[mask]
    y_data = y_data[mask]
    
    initial_guess = [
                        Settings["Data"]["X0"],
                        Settings["Data"]["A0"], Settings["Data"]["TAU0"],
                        Settings["Data"]["A1"], Settings["Data"]["TAU1"],
                        Settings["Data"]["A2"], Settings["Data"]["TAU2"],
                        Settings["Data"]["A3"], Settings["Data"]["TAU3"],
                    ]
    boundlist = [
                    [-100,0,0,0,0,0,0,0,0],
                    [100,10*Settings["Data"]["A0"],10*Settings["Data"]["TAU0"],10*Settings["Data"]["A1"],
                        10*Settings["Data"]["TAU1"],10*Settings["Data"]["A2"],10*Settings["Data"]["TAU2"],
                        10*Settings["Data"]["A3"],10*Settings["Data"]["TAU3"]]
                ]

    params, _ = curve_fit(Exponential_Decay_X, x_data, y_data, p0=initial_guess,bounds=boundlist, maxfev=2000000)

    fit_label = f'Fit: A0={params[1]:.2f}, TAU0={params[2]:.2f}\nA1={params[3]:.2f}, TAU1={params[4]:.2f}\nA2={params[5]:.2f}, TAU2={params[6]:.2f}\nA3={params[7]:.2f}, TAU3={params[8]:.2f}'

    plt.hist(data,bins=np.arange(min(data),max(data)+Settings["Data"]["Bin Width"],Settings["Data"]["Bin Width"]))
    plt.plot(x_data,Exponential_Decay_X(x_data,*params),'-r',label=fit_label)
    plt.yscale('log')
    plt.axvline(x=Settings["Data"]["Fit Start"],c='b')
    plt.legend(loc='best', fontsize='small', title='Parameters\n')